In [ ]:
from fastai.vision.all import *

In [ ]:
#path = "data/"
path = "data/mini/"

In [ ]:
fungi = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=123),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224)
)

In [ ]:
dls = fungi.dataloaders(path)

In [ ]:
dls.show_batch()

In [ ]:
# Find optimal learning rate
# https://docs.fast.ai/callback.schedule.html#Learner.lr_find
# https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html
# learn.lr_find()
# learn.recorder.plot(suggestion=True)
# learn.fine_tune(epochs, lr)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:
interp = ClassificationInterpolation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses()

## Save model

In [ ]:
learn.export('model.pkl')